In [2]:
import pandas as pd
from selenium import webdriver
import time
user_input = ""

while len(user_input) == 0:
    
    user_input = input("What Is The Stock You Want To See Today?")
    
    # ***Scrapping to see if we can find what user entered*** #
    
    # Open Edge brower using Edgedriver
    # This is where you would tell it the location of your path
    path = "C:\Program Files (x86)\msedgedriver.exe"
    
    # Navigate to Yahoo Finance
    driver = webdriver.Edge(path)
    # driver.get opens the url page you want 
    driver.get("https://finance.yahoo.com/")
    time.sleep(3)
    
    # Enter text search bar
    enter_searchbar = driver.find_element_by_id("yfin-usr-qry")
    enter_searchbar.send_keys(user_input)

    # Click on the button to display result
    click_button = driver.find_element_by_id("header-desktop-search-button")
    click_button.click()
    time.sleep(3)
    # Find Title of Company and Ticker Symbol
    try:
        
        name_ticker = driver.find_element_by_xpath('//*[@id="quote-header-info"]/div[2]/div[1]/div[1]/h1').text
        name, ticker = " ".join(name_ticker.split()[:-1]), name_ticker.split()[-1][1:-1]
        driver.quit()
        
    # Answer to question yes or no
        right_company = ""
        
        while len(right_company) == 0:
            
            right_company = input(f"Were you looking for company {name} with the ticker symbol {ticker}? ")
            if right_company.lower() == "yes":
                break
            elif right_company.lower() == "no":
                print("""The Company that you inputted either does not exist or is private. 
                    \n Please input another company""")
                user_input = ""
            else:
                print("You did not put in yes or no. Please input one of those answers")
                right_company = ""
                
    except:
        driver.quit()
        print("""The Company that you inputted either does not exist or is private. 
              \n Please input another company""")
        user_input = ""

# Providing Stock Info for the company the user selected
stock_ans = input("Would you like the stock info also?")

if stock_ans.lower() == "yes":
    driver = webdriver.Edge(path)
    driver.get(f"https://finance.yahoo.com/quote/{ticker}?p={ticker}&.tsrc=fin-srch")
    time.sleep(3)
    raw_stock_data = driver.find_element_by_xpath('//*[@id="quote-summary"]').text
    raw_stock_data = raw_stock_data.split("\n")[:16]
    raw_stock_data = [data.split(" ") for data in raw_stock_data]
    driver.quit()
    organized_data = {
        "Previous Close": " ".join(raw_stock_data[0][2::]), 
        "Open": " ".join(raw_stock_data[1][1:]), 
        "Bid": " ".join(raw_stock_data[2][1:]),
        "Ask": " ".join(raw_stock_data[3][1:]),
        "Day's Range": " ".join(raw_stock_data[4][2:]),
        "52 Week Range": " ".join(raw_stock_data[5][3:]),
        "Volume": "".join(raw_stock_data[6][-1]),
        "Avg. Volume": " ".join(raw_stock_data[7][2:]),
        "Market Cap": "".join(raw_stock_data[8][-1]),
        "Beta(5Y Monthly)": "".join(raw_stock_data[9][-1]),
        "PE Ratio(TTM)": "".join(raw_stock_data[10][-1]),
        "EPS (TTM)": "".join(raw_stock_data[11][-1]),
        "Earning Date": " ".join(raw_stock_data[12][2:]), 
        "Forward Dividend & Yield": " ".join(raw_stock_data[13][-2:]),
        "Ex-Dividend Data": " ".join(raw_stock_data[14][2::]),
        "1Y Target Est": "".join(raw_stock_data[15][-1]),
    }
    
    finalize_data = pd.DataFrame(organized_data, index = [0])
    display(finalize_data)

else:
    print("Ok, See Ya Later!")

,Previous Close,Open,Bid,Ask,Day's Range,52 Week Range,Volume,Avg. Volume,Market Cap,Beta(5Y Monthly),PE Ratio(TTM),EPS (TTM),Earning Date,Forward Dividend & Yield,Ex-Dividend Data,1Y Target Est
0,144.11,144.75,144.32 x 800,144.36 x 1000,142.65 - 145.00,105.92 - 152.84,"3,844,847","3,927,904",129.222B,1.18,24.37,5.92,"Oct 20, 2021",6.56 (4.58%),"Aug 09, 2021",151.67
